<a href="https://colab.research.google.com/github/aeyage/intraday_prices/blob/main/intraday_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import time
import numpy as np
import pandas as pd
import cudf
import cupy as cp

First, we define a function to load our price data from a CSV file using pandas. The function will read the data, set the date as the index, and ensure all date indices are in the correct datetime format.

In [3]:
def get_prices_as_pandas(prices_file):
  d = pd.read_csv(prices_file)
  d = d.set_index("date_time", inplace=True)
  d.index = pd.to_datetime(d.index)

  return d.bfill().ffill()

Next, we define a similar function to load the price data using cuDF, a GPU-accelerated library similar to pandas. This will allow us to perform computations on the GPU.

In [4]:
def get_prices_as_cudf(prices_file):
  c = pd.read_csv(prices_file)
  c = c.set_index("date_time", inplace=True)
  c.index = pd.to_datetime(c.index)

  return c.bfill().ffill()